In [1]:
import pandas as pd
from datetime import datetime
import sql_functions as sql_int

In [2]:
def new_tournament(tournament_add,name,short_name,start,fin):
    start = datetime.strptime(start, '%Y-%m-%d')
    fin = datetime.strptime(fin, '%Y-%m-%d')
    tournament_add.append([name,short_name,start,fin])
def add_tournament(tournament_lst,tournament_df):
    df_add = pd.DataFrame(tournament_lst,columns=['name','short_name','start','fin'])
    df_add.drop_duplicates(inplace=True)
    tournament_df = pd.concat([tournament_df,df_add],ignore_index=True)
    return tournament_df

## New tournament
#### Read new tournaments from CSV-file

In [3]:

tournament = pd.read_csv('/program/tft_overview/tft_Tables/data_sheets/tournaments.csv',
                            dtype={'name' : 'str', 'short_name' : 'str' },
                            parse_dates=["start", "fin"])
df_tournament = tournament.copy()
#df_tournament[['name','short_name']] = df_tournament[['name','short_name']].apply(lambda x : x.str.lower())

FileNotFoundError: [Errno 2] No such file or directory: '/program/tft_overview/tft_Tables/data_sheets/tournaments.csv'

#### Example for new tournament

In [4]:
tournament_add = []

In [5]:
new_tournament(tournament_add,
               "A1 ESPORTS Prismatic Legends REMIX RUMBLE QUALIFIER 1",
               'A1 PL ReRu OQ1',
               '2023-12-09',
               '2023-12-09'
               )

In [6]:
df_tournament = pd.DataFrame()

In [7]:
df_tournament = add_tournament(tournament_add,df_tournament)
#df_tournament[['name','short_name']] = df_tournament[['name','short_name']].apply(lambda x : x.str.lower())

In [8]:
df_tournament

,name,short_name,start,fin
0,A1 ESPORTS Prismatic Legends REMIX RUMBLE QUAL...,A1 PL ReRu OQ1,2023-12-09,2023-12-09


## Get existing Tournaments

In [9]:
schema = 'tft_table'
table = 'tournament'
sql_query = f'''
            select * from {table};
            '''

existing_tournament = pd.DataFrame(sql_int.get_data_alchemy(sql_query),
                                  columns=['tournament_id','name','short_name','start','fin'])

df_existing_tournament = existing_tournament.copy()
df_existing_tournament[['start','fin']] = df_existing_tournament[['start','fin']].apply(pd.to_datetime)
df_existing_tournament['existing'] = 'Yes'
#df_existing_tournament[['name','short_name']] = df_existing_tournament[['name','short_name']].apply(lambda x : x.str.lower())
df_tournament = pd.concat([df_existing_tournament,df_tournament],ignore_index=True)

In [10]:
df_tournament[df_tournament.duplicated(keep = False, subset=['name']) == True].sort_values(by=['name','short_name'])

,tournament_id,name,short_name,start,fin,existing


In [11]:
df_tournament = (df_tournament
                  .drop_duplicates(subset=['name'],keep=False)
                  .query('existing != "Yes"')
                  .drop(['tournament_id','existing'],axis=1)
                  )

In [12]:
df_tournament

,name,short_name,start,fin
22,A1 ESPORTS Prismatic Legends REMIX RUMBLE QUAL...,A1 PL ReRu OQ1,2023-12-09,2023-12-09


push new tournaments into Database

In [13]:
sql_int.push_to_database(df_tournament,table,sql_int.get_engine_alchemy(),schema)

The tournament table was imported successfully.
